In [1]:
%run stdPackages.ipynb

# Read database

In [2]:
from pyDatabases.simpleDB_wheels import read, readSets, broadcast
kwargs = {'variables': ['Fundamentals', 'LoadVariables', 'TransmissionLines', 'GeneratorsVariables','StorageVariables','Regulation'],
        'maps': ['LoadMaps','GeneratorsMaps','StorageMaps'],
        'variable2D': ['HourlyVariation'],
        'scalars': ['Scalars']}
db = read.dbFromWB(os.path.join(d['data'],'mBasicPH_storageLarge.xlsx'), kwargs)
readSets(db)

This model takes roughly 45 seconds to solve. This primarily comes from: 
1. ```self.initBlocks``` (16 seconds)
2. ```self.compileParameters``` (6-7 seconds)
3. ```self.getDenseArgs``` (8 seconds)
4. ```self.postSolve``` (roughly 6 seconds)

Solution time requires roughly 5 seconds.

In [3]:
import time
times = {}
t0 = time.time()
# Initialize model:
m = mBasicPH_storage.mSimple(db) # initialize using the database 'db' from above.
times['setup'] = time.time()-t0
# Initialize blocks:
m.preSolve()
m.initBlocks()
times['initBlocks'] = time.time()-t0-sum(times.values())
# Compile model:
m.blocks.compileParameters()
times['compileParameters'] = time.time()-t0-sum(times.values())
m.blocks.settingsFromCompiled()
times['settingsFromCompiled'] = time.time()-t0-sum(times.values())
m.blocks.inferGlobalDomains()
times['inferGlobalDomains'] = time.time()-t0-sum(times.values())
m.blocks.getDenseArgs()
times['getDenseArgs'] = time.time()-t0-sum(times.values())
args = m.blocks.lp_args
times['lp_args'] = time.time()-t0-sum(times.values())
sol = optimize.linprog(method = m.method, **args)
times['sol'] = time.time()-t0-sum(times.values())
m.postSolve(sol)
times['postSolve'] = time.time()-t0-sum(times.values())
times

{'setup': 0.4917106628417969,
 'initBlocks': 16.33599352836609,
 'compileParameters': 6.791440963745117,
 'settingsFromCompiled': 0.0,
 'inferGlobalDomains': 2.353090286254883,
 'getDenseArgs': 7.825849294662476,
 'lp_args': 2.8059284687042236,
 'sol': 5.606281042098999,
 'postSolve': 6.033329248428345}